<a href="https://colab.research.google.com/github/yukinaga/hopfield_boltzmann/blob/main/section_3/04_rbm_operation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 制限ボルツマンマシンの動作
RestrictedBoltzmannMachineクラスを使い、制限ボルツマンマシンの動作を確認します。

## RestrictedBoltzmannMachineクラス

In [ ]:
import numpy as np

class RestrictedBoltzmannMachine:
    def __init__(self, n_visible, n_hidden, learning_rate=0.1, temperature=1.0, epochs=1000):
        """RBMの初期化"""
        self.n_visible = n_visible
        self.n_hidden = n_hidden
        self.learning_rate = learning_rate
        self.temperature = temperature  # 温度パラメータ
        self.epochs = epochs

        # 重み行列の初期化（標準正規分布で初期化）
        self.weights = np.random.randn(n_hidden, n_visible) * 0.1
        # バイアスの初期化
        self.visible_bias = np.zeros(n_visible)
        self.hidden_bias = np.zeros(n_hidden)

    def sigmoid(self, x):
        """温度パラメータを考慮したシグモイド関数"""
        return 1 / (1 + np.exp(-x / self.temperature))

    def sample_hidden(self, visible):
        """可視層から隠れ層へのサンプリング"""
        activation = np.dot(visible, self.weights.T) + self.hidden_bias
        prob_hidden = self.sigmoid(activation)
        hidden_state = (np.random.rand(self.n_hidden) < prob_hidden).astype(np.int_)
        return prob_hidden, hidden_state

    def sample_visible(self, hidden):
        """隠れ層から可視層へのサンプリング"""
        activation = np.dot(hidden, self.weights) + self.visible_bias
        prob_visible = self.sigmoid(activation)
        visible_state = (np.random.rand(self.n_visible) < prob_visible).astype(np.int_)
        return prob_visible, visible_state

    def contrastive_divergence(self, input_data):
        """Contrastive Divergence法でRBMを学習"""
        prob_hidden, hidden_state = self.sample_hidden(input_data)
        prob_visible, visible_state = self.sample_visible(hidden_state)
        prob_hidden_reconstructed, _ = self.sample_hidden(visible_state)

        # 勾配計算と重み更新
        positive_grad = np.outer(prob_hidden, input_data)
        negative_grad = np.outer(prob_hidden_reconstructed, visible_state)

        self.weights += self.learning_rate * (positive_grad - negative_grad)
        self.visible_bias += self.learning_rate * (input_data - visible_state)
        self.hidden_bias += self.learning_rate * (prob_hidden - prob_hidden_reconstructed)

    def train(self, data):
        """RBMのトレーニング"""
        for epoch in range(self.epochs):
            for sample in data:
                self.contrastive_divergence(sample)

            # 温度を徐々に下げる（アニーリング）
            self.temperature *= 0.99

            if (epoch + 1) % 100 == 0:
                print(f"Epoch {epoch + 1}/{self.epochs}, Temperature: {self.temperature:.4f}")

    def run_hidden(self, data):
        """可視層から隠れ層の表現を取得"""
        _, hidden_state = self.sample_hidden(data)
        return hidden_state

    def run_visible(self, hidden):
        """隠れ層から可視層を再構成"""
        _, visible_state = self.sample_visible(hidden)
        return visible_state

## RestrictedBoltzmannMachineクラスの利用
制限ボルツマンマシンを訓練し、パターンを再現します。

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist

def plot_image(vector, title, shape=(28, 28)):
    """1次元配列を画像として表示"""
    plt.imshow(vector.reshape(shape), cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

# MNISTデータセットの読み込み
(train_X, _), (test_X, _) = mnist.load_data()
train_X = (train_X / 255.0).reshape(-1, 784)  # 正規化と形状の変換
test_X = (test_X / 255.0).reshape(-1, 784)

# RBMの初期化と学習
rbm = RestrictedBoltzmannMachine(n_visible=784, n_hidden=64, learning_rate=0.1, temperature=5.0, epochs=10)
rbm.train(train_X[:1000])  # トレーニングデータの一部（1000サンプル）で学習

# テストデータから1つのサンプルを再構成
sample = test_X[0]
hidden_state = rbm.run_hidden(sample)
reconstructed = rbm.run_visible(hidden_state)

# 元の画像と再構成画像の比較
plot_image(sample, "Original Image")
plot_image(reconstructed, "Reconstructed Image")

以下、上記のコードを解説します。  

---

### **全体の流れ**
1. **データの読み込みと前処理**
   - MNIST データセットを使用します。
   - 画像データを正規化して、RBM が扱いやすい形式に変換します。
  
2. **RBM の初期化と学習**
   - RBM クラスのインスタンスを作成し、学習データを用いてトレーニングします。
   - Contrastive Divergence（CD）を使用して RBM の重みとバイアスを更新します。

3. **再構成**
   - 学習済みの RBM を使って、テストデータの再構成を行い、元の画像と再構成画像を比較します。
  
4. **結果の可視化**
   - 再構成された画像をプロットして、RBM の学習結果を確認します。

---

### **ライブラリのインポート**
```python
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
import numpy as np
```
- **matplotlib**: 画像を可視化するためのライブラリ。
- **tensorflow.keras.datasets**: MNIST データセットを簡単に読み込むために使用。
- **numpy**: 配列の操作や数値計算に使用。

---

### **画像表示関数 (`plot_image` メソッド)**
```python
def plot_image(vector, title, shape=(28, 28)):
    """1次元配列を画像として表示"""
    plt.imshow(vector.reshape(shape), cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()
```
- **目的**: 1次元配列として保存されている画像データを 28x28 の形式に変換して表示します。
- **引数**:
  - `vector`: 784次元のベクトル（28×28ピクセルの画像を一次元化したもの）。
  - `title`: 画像のタイトル。
  - `shape`: 画像の形状（デフォルトでは 28×28 ピクセル）。
- **処理**:
  - `vector.reshape(shape)` で画像データを 2 次元に変換。
  - `plt.imshow()` で画像をグレースケール表示。
  - タイトルを設定し、軸を非表示にして見やすくしています。

---

### **データの読み込みと前処理**
```python
(train_X, _), (test_X, _) = mnist.load_data()
train_X = (train_X / 255.0).reshape(-1, 784)  # 正規化と形状の変換
test_X = (test_X / 255.0).reshape(-1, 784)
```

- **`mnist.load_data()`**: MNIST データセットを読み込みます。
  - `train_X`: トレーニング用の画像データ（60,000 枚の手書き数字画像）。
  - `test_X`: テスト用の画像データ（10,000 枚の手書き数字画像）。
- **正規化**:
  - **`train_X / 255.0`**: ピクセル値（0〜255）を 0〜1 の範囲に正規化します。
- **形状の変換**:
  - **`.reshape(-1, 784)`**: 28x28 の画像を 784 次元のベクトルに変換します。
  
---

### **RBM の初期化と学習**
```python
rbm = RestrictedBoltzmannMachine(n_visible=784, n_hidden=64, learning_rate=0.1, temperature=5.0, epochs=10)
rbm.train(train_X[:1000])  # トレーニングデータの一部（1000サンプル）で学習
```

- **RBM の初期化**:
  - `n_visible=784`: 可視層のノード数（28×28ピクセル）。
  - `n_hidden=64`: 隠れ層のノード数（特徴の数）。
  - `learning_rate=0.1`: 重みの更新時の学習率。
  - `temperature=5.0`: 初期温度パラメータ。
  - `epochs=10`: エポック数（学習の繰り返し回数）。
  
- **トレーニング**:
  - `train_X[:1000]` で、トレーニングデータから 1000 サンプルを使用して学習します。
  - `train()` メソッドを呼び出して、Contrastive Divergence を用いて重みとバイアスを更新します。

---

### **再構成と表示**
```python
sample = test_X[0]
hidden_state = rbm.run_hidden(sample)
reconstructed = rbm.run_visible(hidden_state)
```

- **テストデータから 1 つの画像を再構成**:
  - **`sample`**: テストデータの最初のサンプルを使用。
  - **`run_hidden()`**: 可視層の入力から隠れ層の特徴ベクトルを生成。
  - **`run_visible()`**: 隠れ層の特徴から可視層の再構成を行います。

```python
plot_image(sample, "Original Image")
plot_image(reconstructed, "Reconstructed Image")
```
- **元の画像と再構成された画像を表示**:
  - `plot_image()` 関数を用いて、元の画像と再構成画像をそれぞれ表示します。

---

### **実行結果の期待**
- **Original Image**: テストデータから取り出した手書き数字の画像が表示されます。
- **Reconstructed Image**: 学習済みの RBM によって再構成された画像が表示されます。

---

### **今後の可能性**
このコードを基にして、以下のような発展が可能です：
- 隠れ層のノード数や温度パラメータを調整して、再構成の品質を向上させる。
- より多くのエポックで学習させ、再構成の精度を高める。